In [1]:
import requests
from sqlalchemy import create_engine
import pandas as pd
import geopandas as gpd
import os

In [ ]:
engine = create_engine(   
"mssql+pyodbc:///?odbc_connect="
"Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B"
"Server%3Drliddb.int.lcog.org%2C5433%3B"
"Database%3DRLIDGeo%3B"
"Trusted_Connection%3Dyes%3B"
"ApplicationIntent%3DReadWrite%3B"
"WSID%3Dclwrk4087.int.lcog.org%3B")

zip_sql = '''
SELECT 
zipcode AS ZipCode,
mailcity AS City,
Shape.STAsBinary() AS geometry
FROM dbo.ZIPCode;
'''

In [2]:
inpath = r'T:\DCProjects\Support\Lane\TransitSurvey'

In [6]:
dat = pd.read_excel(os.path.join(inpath, 'RAW DATA Transit Planning for Rural Lane County.xlsx'), 
                    sheet_name = 'OriginDestination', header=None)

In [8]:
dat.columns = ['Origin', 'Destination']